In [2]:
import tkinter as tk
from tkinter import messagebox
import sqlite3

class Seat:
    def __init__(self, row, number, status="available"):
        self.row = row
        self.number = number
        self.status = status

    def set_status(self, status):
        self.status = status

    def is_available(self):
        return self.status == "available"

class Movie:
    def __init__(self, title, showtimes):
        self.title = title
        self.showtimes = showtimes

    def get_title(self):
        return self.title

    def get_showtimes(self):
        return self.showtimes

class Showtime:
    def __init__(self, movie, time, seats):
        self.movie = movie
        self.time = time
        self.seats = seats  # 2D list of Seat objects

    def get_time(self):
        return self.time

    def get_seats(self):
        return self.seats

class Booking:
    def __init__(self, movie, showtime, seat):
        self.movie = movie
        self.showtime = showtime
        self.seat = seat

class DatabaseManager:
    def __init__(self, db_name):
        self.conn = sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()
        self.create_tables()
        self.initialize_data()

    def create_tables(self):
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS movies
                              (title TEXT PRIMARY KEY)''')
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS showtimes
                              (movie_title TEXT, time TEXT)''')
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS bookings
                              (movie_title TEXT, showtime TEXT, row INT, seat_num INT)''')
        self.conn.commit()

    def initialize_data(self):
        # Add sample movies and showtimes if not already present
        movies = ["Avengers", "Inception"]
        for movie in movies:
            self.cursor.execute("INSERT OR IGNORE INTO movies (title) VALUES (?)", (movie,))
        self.conn.commit()

        showtimes = [("Avengers", "10:00 AM"), ("Avengers", "2:00 PM"), ("Inception", "12:00 PM"), ("Inception", "6:00 PM")]
        for movie, time in showtimes:
            self.cursor.execute("INSERT OR IGNORE INTO showtimes (movie_title, time) VALUES (?, ?)", (movie, time))
        self.conn.commit()

    def get_movies(self):
        self.cursor.execute("SELECT title FROM movies")
        return [row[0] for row in self.cursor.fetchall()]

    def get_showtimes(self, movie_title):
        self.cursor.execute("SELECT time FROM showtimes WHERE movie_title = ?", (movie_title,))
        return [row[0] for row in self.cursor.fetchall()]

    def save_booking(self, booking):
        self.cursor.execute("INSERT INTO bookings VALUES (?, ?, ?, ?)",
                           (booking.movie.title, booking.showtime.time, booking.seat.row, booking.seat.number))
        self.conn.commit()

    def load_bookings(self, movie_title, showtime):
        self.cursor.execute("SELECT row, seat_num FROM bookings WHERE movie_title = ? AND showtime = ?",
                           (movie_title, showtime))
        return [(row[0], row[1]) for row in self.cursor.fetchall()]

    def close_connection(self):
        self.conn.close()

class CinemaApp:
    def __init__(self):
        self.root = tk.Tk()  # Create the Tkinter window
        self.root.title("Panaview Cinemas Booking System")
        self.root.geometry("800x600")
        self.root.protocol("WM_DELETE_WINDOW", self.close_app)
        self.db = DatabaseManager("cinema.db")
        self.seats = {}
        self.selected_seats = []
        self.create_gui()

    def create_gui(self):
        self.movie_var = tk.StringVar()
        self.showtime_var = tk.StringVar(value="")
        tk.Label(self.root, text="Select Movie:", font=("Arial", 12)).pack(pady=10)
        self.movie_menu = tk.OptionMenu(self.root, self.movie_var, *self.db.get_movies(), command=self.on_movie_select)
        self.movie_menu.pack(pady=5)
        tk.Label(self.root, text="Select Showtime:", font=("Arial", 12)).pack(pady=10)
        self.showtime_menu = tk.OptionMenu(self.root, self.showtime_var, "", command=self.on_showtime_select)
        self.showtime_menu.pack(pady=5)
        self.seat_frame = tk.Frame(self.root)
        self.seat_frame.pack(pady=20)
        tk.Button(self.root, text="Confirm Booking", command=self.confirm_booking).pack(pady=10)

    def on_movie_select(self, event):
        self.showtime_var.set("")
        showtimes = self.db.get_showtimes(self.movie_var.get())
        menu = self.showtime_menu["menu"]
        menu.delete(0, tk.END)
        for time in showtimes:
            menu.add_command(label=time, command=lambda t=time: self.showtime_var.set(t))
        self.clear_seat_layout()

    def on_showtime_select(self, event):
        if self.showtime_var.get():
            self.display_seat_layout()

    def display_seat_layout(self):
        self.clear_seat_layout()
        # Create a 5x10 seat grid
        seats = [[Seat(row, col) for col in range(10)] for row in range(5)]
        # Mark booked seats
        booked_seats = self.db.load_bookings(self.movie_var.get(), self.showtime_var.get())
        for row, col in booked_seats:
            seats[row][col].set_status("booked")
        # Display seats
        for row in range(5):
            for col in range(10):
                seat = seats[row][col]
                bg_color = "green" if seat.is_available() else "red"
                btn = tk.Button(self.seat_frame, text=f"{row+1}-{col+1}", bg=bg_color,
                                command=lambda r=row, c=col: self.on_seat_click(r, c))
                btn.grid(row=row, column=col, padx=2, pady=2)
                self.seats[(row, col)] = btn

    def on_seat_click(self, row, number):
        seat = self.seats[(row, number)]
        if seat["bg"] == "green":
            seat.config(bg="yellow")
            self.selected_seats.append((row, number))
        elif seat["bg"] == "yellow":
            seat.config(bg="green")
            self.selected_seats.remove((row, number))
        else:
            messagebox.showerror("Error", "This seat is already booked!")

    def confirm_booking(self):
        if not self.movie_var.get() or not self.showtime_var.get() or not self.selected_seats:
            messagebox.showerror("Error", "Please select a movie, showtime, and at least one seat.")
            return
        for row, num in self.selected_seats:
            movie = Movie(self.movie_var.get(), [])
            showtime = Showtime(movie, self.showtime_var.get(), [])
            booking = Booking(movie, showtime, Seat(row, num, "booked"))
            self.db.save_booking(booking)
            self.seats[(row, num)].config(bg="red")
        messagebox.showinfo("Success", "Booking confirmed!")
        self.selected_seats.clear()

    def clear_seat_layout(self):
        for widget in self.seat_frame.winfo_children():
            widget.destroy()
        self.seats.clear()

    def close_app(self):
        self.db.close_connection()
        self.root.destroy()

    def run(self):
        self.root.mainloop()

if __name__ == "__main__":
    app = CinemaApp()
    app.run()

In [4]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
from datetime import datetime

class Seat:
    def __init__(self, row, number, status="available"):
        self.row = row
        self.number = number
        self.status = status

    def set_status(self, status):
        self.status = status

    def is_available(self):
        return self.status == "available"

    def get_status(self):
        return self.status

class Booking:
    def __init__(self, movie_title, showtime, seat):
        self.movie_title = movie_title
        self.showtime = showtime
        self.seat = seat

class DatabaseManager:
    def __init__(self, db_name):
        self.conn = sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()
        self.create_tables()

    def create_tables(self):
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS movies
                              (title TEXT PRIMARY KEY, duration INT)''')
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS showtimes
                              (movie_title TEXT, showtime TEXT, 
                               FOREIGN KEY (movie_title) REFERENCES movies(title))''')
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS bookings
                              (movie_title TEXT, showtime TEXT, row INT, seat_num INT,
                               timestamp TEXT,
                               PRIMARY KEY (movie_title, showtime, row, seat_num))''')
        self.conn.commit()

    def add_movie(self, title, duration):
        self.cursor.execute("INSERT OR IGNORE INTO movies VALUES (?, ?)", (title, duration))
        self.conn.commit()

    def add_showtime(self, movie_title, showtime):
        self.cursor.execute("INSERT OR IGNORE INTO showtimes VALUES (?, ?)", (movie_title, showtime))
        self.conn.commit()

    def get_movies(self):
        self.cursor.execute("SELECT title FROM movies")
        return [row[0] for row in self.cursor.fetchall()]

    def get_showtimes(self, movie_title):
        self.cursor.execute("SELECT showtime FROM showtimes WHERE movie_title = ?", (movie_title,))
        return [row[0] for row in self.cursor.fetchall()]

    def save_booking(self, booking):
        self.cursor.execute("INSERT INTO bookings VALUES (?, ?, ?, ?, ?)",
                           (booking.movie_title, booking.showtime, booking.seat.row, 
                            booking.seat.number, datetime.now().isoformat()))
        self.conn.commit()

    def get_booked_seats(self, movie_title, showtime):
        self.cursor.execute("SELECT row, seat_num FROM bookings WHERE movie_title = ? AND showtime = ?",
                           (movie_title, showtime))
        return [(row[0], row[1]) for row in self.cursor.fetchall()]

    def close_connection(self):
        self.conn.close()

class CinemaApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Panaview Cinemas Booking System")
        self.root.geometry("800x600")
        self.root.protocol("WM_DELETE_WINDOW", self.close_app)
        self.db = DatabaseManager("cinema.db")
        self.seats = {}  # (row, col): Button
        self.selected_seats = []
        self.seat_layout = []  # 2D list of Seat objects
        self.initialize_data()
        self.create_gui()

    def initialize_data(self):
        # Sample data for testing
        self.db.add_movie("Avengers: Endgame", 180)
        self.db.add_movie("The Matrix", 136)
        self.db.add_showtime("Avengers: Endgame", "10:00 AM")
        self.db.add_showtime("Avengers: Endgame", "2:00 PM")
        self.db.add_showtime("The Matrix", "12:00 PM")
        self.db.add_showtime("The Matrix", "4:00 PM")

    def create_gui(self):
        # Movie selection
        tk.Label(self.root, text="Select Movie:", font=("Arial", 12)).pack(pady=10)
        self.movie_var = tk.StringVar()
        self.movie_menu = tk.OptionMenu(self.root, self.movie_var, *self.db.get_movies(),
                                        command=self.on_movie_select)
        self.movie_menu.pack()

        # Showtime selection
        tk.Label(self.root, text="Select Showtime:", font=("Arial", 12)).pack(pady=10)
        self.showtime_var = tk.StringVar()
        self.showtime_menu = tk.OptionMenu(self.root, self.showtime_var, [],
                                          command=self.on_showtime_select)
        self.showtime_menu.pack()

        # Seat layout
        self.seat_frame = tk.Frame(self.root)
        self.seat_frame.pack(pady=20)

        # Legend for seat status
        tk.Label(self.root, text="Available: Green | Selected: Yellow | Booked: Red",
                 font=("Arial", 10)).pack(pady=5)

        # Confirm button
        tk.Button(self.root, text="Confirm Booking", command=self.confirm_booking,
                  font=("Arial", 12)).pack(pady=10)

    def on_movie_select(self, event):
        self.showtime_var.set("")
        self.clear_seat_layout()
        showtimes = self.db.get_showtimes(self.movie_var.get())
        menu = self.showtime_menu["menu"]
        menu.delete(0, tk.END)
        for time in showtimes:
            menu.add_command(label=time, command=lambda t=time: self.showtime_var.set(t))

    def on_showtime_select(self, event):
        if self.showtime_var.get():
            self.display_seat_layout()

    def display_seat_layout(self):
        self.clear_seat_layout()
        self.seat_layout = [[Seat(row, col) for col in range(10)] for row in range(5)]
        booked_seats = self.db.get_booked_seats(self.movie_var.get(), self.showtime_var.get())
        
        # Mark booked seats
        for row, col in booked_seats:
            self.seat_layout[row][col].set_status("booked")

        # Create seat buttons
        for row in range(5):
            for col in range(10):
                seat = self.seat_layout[row][col]
                bg_color = "green" if seat.is_available() else "red"
                btn = tk.Button(self.seat_frame, text=f"{chr(65+row)}{col+1}", width=4, height=2,
                               bg=bg_color, command=lambda r=row, c=col: self.on_seat_click(r, c))
                btn.grid(row=row, column=col, padx=2, pady=2)
                self.seats[(row, col)] = btn

    def on_seat_click(self, row, number):
        seat = self.seat_layout[row][number]
        button = self.seats[(row, number)]
        if not seat.is_available():
            messagebox.showerror("Error", "This seat is already booked!")
            return
        if button["bg"] == "green":
            button.config(bg="yellow")
            seat.set_status("selected")
            self.selected_seats.append((row, number))
        elif button["bg"] == "yellow":
            button.config(bg="green")
            seat.set_status("available")
            self.selected_seats.remove((row, number))

    def confirm_booking(self):
        if not self.movie_var.get() or not self.showtime_var.get():
            messagebox.showerror("Error", "Please select a movie and showtime!")
            return
        if not self.selected_seats:
            messagebox.showerror("Error", "Please select at least one seat!")
            return
        for row, num in self.selected_seats:
            seat = self.seat_layout[row][num]
            seat.set_status("booked")
            self.seats[(row, num)].config(bg="red")
            booking = Booking(self.movie_var.get(), self.showtime_var.get(), seat)
            self.db.save_booking(booking)
        messagebox.showinfo("Success", "Booking confirmed!")
        self.selected_seats.clear()

    def clear_seat_layout(self):
        for widget in self.seat_frame.winfo_children():
            widget.destroy()
        self.seats.clear()
        self.selected_seats.clear()
        self.seat_layout = []

    def close_app(self):
        self.db.close_connection()
        self.root.destroy()

    def run(self):
        self.root.mainloop()

if __name__ == "__main__":
    app = CinemaApp()
    app.run()

OperationalError: table movies has 1 columns but 2 values were supplied